# Подготовка библиотек

### Скачивание библиотек

In [ ]:
!pip install datasets zstandard jsonlines pysimdjson

### Установка библиотек

In [ ]:
from datasets import load_dataset
from transformers import GPT2LMHeadModel, GPT2TokenizerFast, GPT2Config
from transformers import get_linear_schedule_with_warmup

import torch
from torch.optim import AdamW
from torch.utils.data import Dataset, DataLoader
from torch.utils.data import random_split, RandomSampler, SequentialSampler

import pandas as pd
model_save_path = './model'

# Подготовка датасета

In [ ]:
dataset = load_dataset('IlyaGusev/ru_news', split="train", streaming=True, trust_remote_code=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
texts = []
titles = []
test_text = []
test_titles = []
# Выбираю данные из датасета
for idx, i in enumerate(dataset.take(600)):
  if idx >= 500:
    test_text.append(i['text'].replace('\n', ' '))
    test_titles.append(i['title'])
  else:
    texts.append(i['text'].replace('\n', ' '))
    titles.append(i['title'])

In [ ]:
df = pd.DataFrame({
    'text' : texts,
    'title' : titles
})

In [ ]:
# Добавляю токен начала и конца последовательности
def form_string(ingredient,instruction):
    s = f"<start>Описание: {ingredient.strip()} " \
        f"Заголовок: {instruction.strip()}<end>"
    return s

data = df.apply(lambda x:form_string(
    x['text'], x['title']), axis=1).to_list()

# Токенизатор

In [ ]:
model_name = "sberbank-ai/rugpt3medium_based_on_gpt2"
# Обозначаю ключевые токены
tokenizer = GPT2TokenizerFast.from_pretrained(model_name,
                                             bos_token='<start>',
                                             eos_token='<end>',
                                             unk_token='<unk>',
                                             pad_token='<pad>'
                                             )


In [ ]:
batch_size = 2
max_length = 180

# создаю датасет
class RecipeDataset(Dataset):
    def __init__(self, data, tokenizer):
        self.data = data
        self.input_ids = []
        self.attn_masks = []

        for recipe in data:
            encodings = tokenizer.encode_plus(recipe, # строка
                                              truncation=True, # усечение последовательностей
                                              padding='max_length', # дополнять последовательность по длине
                                              max_length=max_length, # максимальная длина
                                              return_tensors='pt' # вернуть в виде pytorch тензора
                                             )
            self.input_ids.append(torch.squeeze(encodings['input_ids'],0)) # размерность [1,180] -> [180]
            self.attn_masks.append(torch.squeeze(encodings['attention_mask'],0))


    def __len__(self):
        return len(self.data)

    def __getitem__(self,idx):
        return self.input_ids[idx], self.attn_masks[idx]

dataset = RecipeDataset(data, tokenizer)

In [ ]:
# Разделение на обучающую и валидационную выборку
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size

train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# Разбиваем датасет на батчи
train_dataloader = DataLoader(
            train_dataset, # Датасет
            sampler = RandomSampler(train_dataset), # Перемешивание
            batch_size = batch_size # Размер батча
        )

# Валидация
validation_dataloader = DataLoader(
            val_dataset,
            sampler = SequentialSampler(val_dataset),
            batch_size = batch_size
        )

In [ ]:
device = "cuda" # GPU
configuration = GPT2Config.from_pretrained(model_name) # конфигурация
model = GPT2LMHeadModel.from_pretrained(model_name, config=configuration) # подготовка модели
model = model.to(device) # использовать модель на ГПУ
model.resize_token_embeddings(len(tokenizer)) # изменение пространства ембеддинга

epochs = 10
learning_rate = 2e-5
warmup_steps = 1e2

epsilon = 1e-8 # для предотвращения деления на 0 (для числовой стабильности)
optim = AdamW(model.parameters(), lr = learning_rate, eps = epsilon) # оптимизатор

total_steps = len(train_dataloader) * epochs  # шаги

# Постепенное уменьшение скорости обучения
scheduler = get_linear_schedule_with_warmup(optim,
                                            num_warmup_steps=warmup_steps,
                                            num_training_steps=total_steps)

config.json:   0%|          | 0.00/761 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.73G [00:00<?, ?B/s]

In [ ]:
# Промпт
def infer(prompt):
    input = f"<start>Описание: {prompt.strip()}"
    input = tokenizer(input, return_tensors="pt")
    input_ids      = input["input_ids"]
    attention_mask = input["attention_mask"]

    output = model.generate(input_ids.to(device),
                            attention_mask=attention_mask.to(device),
                            max_new_tokens=max_length,
                            do_sample = True, top_k = 50, top_p = 0.85)
    output = tokenizer.decode(output[0], skip_special_tokens=True)
    return output

In [ ]:
# Тренировка
# Проход по эпохам
for epoch_i in range(0, epochs):
    total_train_loss = 0
    model.train() # Модель в обучение
    # Проход по батчам
    for step, batch in enumerate(train_dataloader):
        b_input_ids = batch[0].to(device) # Токенизированное предложение
        b_labels    = batch[0].to(device) # Метки
        b_masks     = batch[1].to(device) # Маска внимания

        model.zero_grad() # Обнуление градиентов
        outputs = model( input_ids = b_input_ids, labels = b_labels,
                         attention_mask = b_masks, token_type_ids = None )

        loss = outputs[0]

        loss.backward() # Вычисление градиентов
        optim.step() # Шаг оптимизатора
        scheduler.step() # изменение скорости обучения

In [ ]:
# Сохраняю модель
model.save_pretrained(model_save_path)
tokenizer.save_pretrained(model_save_path)

('./model/tokenizer_config.json',
 './model/special_tokens_map.json',
 './model/vocab.json',
 './model/merges.txt',
 './model/added_tokens.json',
 './model/tokenizer.json')

In [ ]:
# Загружаю модель
model = GPT2LMHeadModel.from_pretrained(model_save_path)
tokenizer = GPT2TokenizerFast.from_pretrained(model_save_path)
model.to(device)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50259, 1024)
    (wpe): Embedding(2048, 1024)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-23): 24 x GPT2Block(
        (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=1024, out_features=50259, bias=False)
)

In [ ]:
def infer(prompt, title):
    print(f"Текст: {prompt}")
    print(f"Верный ответ: {title}\n\n")
    input1 = f"<start>Описание: {prompt.strip()}"
    input = tokenizer(input1, return_tensors="pt")
    input_ids      = input["input_ids"]
    attention_mask = input["attention_mask"]

    output = model.generate(input_ids.to(device),
                            attention_mask=attention_mask.to(device),
                            max_new_tokens=20,
                            do_sample = True, top_k = 10, top_p = 0.5)
    output = tokenizer.decode(output[0], skip_special_tokens=True)
    output = output.replace(f"Описание: {prompt.strip()}", '')
    return output

In [ ]:
infer(test_text[1], test_titles[1])

Текст: На XXXIX Очередном Конгрессе УЕФА в Вене Мишель Платини был переизбран Президентом УЕФА на третий четырехлетний срок. Платини, который был единственным кандидатом, поддержали делегаты всех 54 национальных ассоциаций УЕФА. Французский функционер был избран президентом европейского футбольного союза на XXXI Очередном конгрессе УЕФА в Дюссельдорфе в январе 2007 года. На XXXV Очередном конгрессе УЕФА в Париже его переизбрали на второй срок, пишет сайт УЕФА. Платини поблагодарил европейские национальные ассоциации за доверие, которое они по-прежнему оказывают. "Спасибо... Большое спасибо, - сказал он. - Спасибо вам за доверие и поддержку. Это значит для меня больше, чем вы можете себе представить". "Горжусь тем, что являюсь вашим товарищем по команде и капитаном нашей команды-победительницы. Горжусь всеми нашими достижениями. И с радостью принимаю новые испытания, которые ждут нас в ближайшие четыре года", - добавил Платини. "Знаю, что могу рассчитывать на всех вас, а вы знаете, что 

' Заголовок: Мишель Платини переизбран Президентом УЕФАСентябрь 2007Описание'

In [ ]:
infer(test_text[3], test_titles[3])

Текст: По факту крушения самолета в ХМАО возбуждено уголовное делоФото: Александр Мамаев © URA. Ru Уральское следственное управление на транспорте возбудило уголовное дело по факту крушения гидросамолета в Югре. У сыщиков есть первые версии ЧП. Уголовное дело возбуждено по статье «нарушение правил безопасности движения и эксплуатации воздушного транспорта, повлекшее по неосторожности смерть двух или более лиц». Основные версии — нарушение правил эксплуатации и безопасности полета, сложные метеоусловия и неисправность техники. Напомним, в 27 километрах от Ханты-Мансийска потерпел крушение легкомоторный гидросамолет «Пеликан-2», принадлежащий 60-летнему жителю окружной столицы. За штурвалом находился сам владелец — Владимир Слинкин, который до выхода на пенсию работал командиром Як-40. Пассажиром оказался 72-летний Валерий Климов. Оба пенсионера погибли на месте. Сейчас следователи проводят осмотр места происшествия.
Верный ответ: По факту крушения самолета в ХМАО возбуждено уголовное де

' Заголовок: Самолету авиакомпании «ЮТэйр» в ХМАО грозит крупный штраф'

In [ ]:
infer(test_text[10], test_titles[10])

Текст: На территории Ямальского района в Ямало-Ненецком автономном округе обнаружено тело мамонтенка, говорится в сообщении пресс-службы района, поступившем в редакцию "Ленты. Ру". Мамонтенка в середине августа 2011 года (точную дату находки пресс-служба Ямальского района не приводит) обнаружил бригадир оленеводческой бригады хозяйства "Ярсалинское" Петр Ядне. Сведений о состоянии останков вымершего животного нет. В Ямальском районе не в первый раз обнаруживают останки мамонтов. Одной из наиболее известных находок является хорошо сохранившееся тело мамонтенка, получившего имя Люба. Животное было найдено в мае 2007 года на реке Юрибей. Мамонты вымерли более восьми тысяч лет назад. Точные причины, которые привели к их вымиранию, окончательно не установлены. На территории Сибири и Аляски в вечной мерзлоте неоднократно находили трупы мамонтов, сохранившиеся в хорошем состоянии до наших дней.
Верный ответ: На Ямале нашли мамонтенка




' Заголовок: На Ямале нашли мамонта. ФОТО, ВИДЕО'

In [ ]:
infer(test_text[12], test_titles[12])

Текст: БРЮССЕЛЬ, 20 октября. /ТАСС/. Убийство журналиста Джамаля Хашкаджи в генконсульстве Саудовской Аравии - шокирующее нарушение Венской конвенции о консульских сношениях 1963 года. Преступление вызывает глубочайшую озабоченность и требует независимого расследования, говорится в распространенном в субботу заявлении верховного представителя ЕС по иностранным делам и политике безопасности Федерики Могерини. n"После почти трех недель наконец появились факты, подтверждающие, что журналист Джамаль Хашкаджи был убит в здании генконсульства Саудовской Аравии в Стамбуле 2 октября 2018 года, - заявила дипломат. - Обстоятельства его гибели вызывают глубочайшую озабоченность и являются шокирующим нарушением Венской конвенции о консульских сношениях 1963 года, в особенности ее 55-й статьи". nЭта статья, в частности, обязывает всех консульских работников полностью соблюдать законы страны пребывания. Согласно ее тексту, консульские помещения "не должны использоваться в целях, не совместимых с вып

' Заголовок: Саудовская Аравия и ЕС официально подтвердили, что убили журналистасаудадца'